In [3]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 11.5 MB/s eta 0:00:0000:010:01


In [5]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.3 MB/s eta 0:00:00:00:0100:01


In [6]:
import boto3
from configparser import ConfigParser
import pandas as pd
import psycopg2
from io import BytesIO

In [27]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [28]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_context = config.get("Bucket", "bucket_context")

# Nome do arquivo a ser lido
source_filename = config.get("FILE", "coordenadas_municipios_csv")

In [30]:
print("endpoint: ", endpoint)
print("access_key: ", access_key)
print("secret_key: ", secret_key)
print("bucket_context: ", bucket_context)
print("source_filename: ", source_filename)

endpoint:  http://minio:9000
access_key:  aulafia
secret_key:  aulafia@123
bucket_context:  context
source_filename:  Coordenadas.csv


In [31]:
# Configurar as credenciais do PostgreSQL
postgres_host = config.get("POSTGRESQL", "host_name")
postgres_port = config.get("POSTGRESQL", "port")  # Porta padrão do PostgreSQL
postgres_user = config.get("POSTGRESQL", "user")
postgres_password = config.get("POSTGRESQL", "user_pwd")
postgres_db = config.get("POSTGRESQL", "db_name")

In [32]:
print('postgres_host:', postgres_host)
print('postgres_port:', postgres_port)
print('postgres_user:', postgres_user)
print('postgres_password:', postgres_password)
print('postgres_db:', postgres_db)

postgres_host: postgres
postgres_port: 5432
postgres_user: aulafia
postgres_password: aulafia@123
postgres_db: db_aulafia


In [33]:
# Inicializar o cliente boto3 para S3
minio_client = boto3.client("s3", 
                            endpoint_url=endpoint,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key
)

In [34]:
# Obter o conteúdo do arquivo CSV do MinIO
response = minio_client.get_object(Bucket=bucket_context, Key=source_filename)
csv_content = response['Body'].read()

In [35]:
# Ler o conteúdo do CSV em um DataFrame
data_frame = pd.read_csv(BytesIO(csv_content))

In [36]:
data_frame.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [38]:
# Conectar ao PostgreSQL
postgres_connection = psycopg2.connect(
    host=postgres_host,
    port=postgres_port,
    user=postgres_user,
    password=postgres_password,
    dbname=postgres_db
)

try:
    # Apagar a tabela coordenadas (se existir)
    with postgres_connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS coordenadas")

    # Criar novamente a tabela coordenadas
    with postgres_connection.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE coordenadas (
                codigo_ibge VARCHAR(100),
                nome VARCHAR(100),
                latitude VARCHAR(20),
                longitude VARCHAR(20),
                capital VARCHAR(4),
                codigo_uf VARCHAR(4),
                siafi_id VARCHAR(4),
                ddd VARCHAR(4),
                fuso_horario VARCHAR(32)
            )
        """)
    postgres_connection.commit()

    # Inserir os dados na tabela coordenadas
    with postgres_connection.cursor() as cursor:
        for index, row in data_frame.iterrows():
            sql = "INSERT INTO coordenadas VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            values = (row['codigo_ibge'], row['nome'], row['latitude'], row['longitude'],
                      row['capital'], row['codigo_uf'], row['siafi_id'], row['ddd'], row['fuso_horario'])
            cursor.execute(sql, values)
        postgres_connection.commit()

    print(f'Dados do arquivo {source_filename} inseridos na tabela coordenadas com sucesso.')

finally:
    postgres_connection.close()

Dados do arquivo Coordenadas.csv inseridos na tabela coordenadas com sucesso.


In [39]:
minio_client.close()